In [1]:
import numpy as np
import pandas as pd
import time

keyword = ['서울특별시 중구','서울특별시 종로구','서울특별시 용산구','서울특별시 성동구','서울특별시 광진구',
           '서울특별시 동대문구','서울특별시 중랑구','서울특별시 성북구','서울특별시 강북구','서울특별시 도봉구',
           '서울특별시 노원구','서울특별시 은평구','서울특별시 서대문구','서울특별시 마포구','서울특별시 양천구',
           '서울특별시 강서구','서울특별시 구로구','서울특별시 금천구','서울특별시 영등포구','서울특별시 동작구',
           '서울특별시 관악구','서울특별시 서초구','서울특별시 강남구','서울특별시 송파구','서울특별시 강동구']

for a in range(len(keyword)):
    collectDf = []
    url = 'https://m.land.naver.com/search/result/' + keyword[a] + '#mapFullList'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

    res = requests.get(url, headers=headers, stream=True)
    res.raw.chunked = False
    print(res.url)
    main_url = res.url.split('?')[0]

    lat = main_url.split('/')[4].split(':')[0] 
    lon = main_url.split('/')[4].split(':')[1] 
    z = main_url.split('/')[4].split(':')[2]
    cortarNo = main_url.split('/')[4].split(':')[3] 
    buildingType = 'SG' 
    keyList = ['atclNo', 'atclNm', 'rletTpNm', 'tradTpNm', 'atclCfmYmd', 'flrInfo', 'prc', 'lat', 'lng', 'sameAddrCnt', 'direction','atclFetrDesc','spc2']

    def extractList(page):
        sub_url = 'https://m.land.naver.com/cluster/ajax/articleList?rletTpCd={}&tradTpCd=A1%3AB1%3AB2&z={}&lat={}&lon={}&cortarNo={}&page={}'.format(buildingType, z, lat, lon, cortarNo, page)
        res = requests.get(sub_url, headers=headers).json()
    
        dataList = []

        for i in range(len(res['body'])):
            dataEle = []
            for ele in keyList:
                try:
                    dataEle.append(res['body'][i][ele])
                except:
                    dataEle.append('')
            dataList.append(dataEle)
        df = pd.DataFrame(dataList, columns=keyList)
        return df

    for i in range(1, 1000):
        try:
            df = extractList(i)
            if df.shape[0] > 0:
                collectDf.append(df)
            else:
                print(i)
                break
        except requests.exceptions.RequestException as e:
            print(f"Error occurred while requesting data: {e}")
        time.sleep(5)
    pd.concat(collectDf).to_csv(f'C:/Users/yoon/store_{keyword[a]}.csv', encoding='utf-8-sig')

# 서울 지역별로 저장된 csv 파일들을 하나로 합쳐서 저장
csv_list = []
for a in range(len(keyword)):
    csv = pd.read_csv(f'C:/Users/yoon/store{a}.csv', encoding='utf-8-sig', index_col=0)
    csv_list.append(csv)
merged_csv = pd.concat(csv_list, ignore_index=True)
merged_csv.to_csv('C:/Users/yoon/store_merged.csv', encoding='euc-kr')
    
       

https://m.land.naver.com/map/37.573025:126.979638:12:1111000000/*/A1:B1:B2?imageUrl=https%3A%2F%2Fssl.pstatic.net%2Fstatic.land%2Fstatic%2Fspace%2F2023032017&photoUrl=https%3A%2F%2Flandthumb-phinf.pstatic.net&myhouseUrl=https%3A%2F%2Fneo.land.naver.com&gladSdkUrl=https%3A%2F%2Fssl.pstatic.net%2Ftveta%2Flibs%2Fglad%2Fprod%2Fgfp-core.js&userAgent=Mozilla%2F5.0+%28Windows+NT+10.0%3B+Win64%3B+x64%29+AppleWebKit%2F537.36+%28KHTML%2C+like+Gecko%29+Chrome%2F58.0.3029.110+Safari%2F537.3&isMobile=false&isAndroid=false&isLandApp=false&isLandAndroidApp=false&isLandIosApp=false&rootPath=query#mapFullList
[['2313550036', '일반상가', '상가', '월세', '', 'B1/3', 1000, '', '', 1, '북향', '31', '31']]
[['2313550036', '일반상가', '상가', '월세', '', 'B1/3', 1000, '', '', 1, '북향', '31', '31'], ['2314714701', '단지내상가', '상가', '월세', '', '1/1', 5000, '', '', 1, '남서향', '77', '38.92']]
[['2313550036', '일반상가', '상가', '월세', '', 'B1/3', 1000, '', '', 1, '북향', '31', '31'], ['2314714701', '단지내상가', '상가', '월세', '', '1/1', 5000, '', '', 